In [2]:
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
import matplotlib.pyplot as plt
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import PPO,DDPG,DQN
from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy
import gym
import numpy as np
import time
from stable_baselines3.common.callbacks import EvalCallback
import env
env = env.ElectricVehicleEnv()
#vec_env = make_vec_env(env, n_envs=4)
check_env(env)
env = DummyVecEnv([lambda: env])

# 设置评估频率
eval_freq = 1000  # 每训练500步评估一次

# 配置EvalCallback
eval_callback = EvalCallback(
    env,
    callback_on_new_best=None,
    n_eval_episodes=20,
    best_model_save_path="./best_model/DQN",  # 更改保存路径为DQN
    log_path="./logs/DQN",  # 更改日志路径为DQN
    eval_freq=eval_freq,
)


In [ ]:

# DQN的超参数设置
model = DQN(
    'MlpPolicy', 
    env, 
    verbose=0, 
    tensorboard_log="./logs/DQN",
    learning_rate=0.0005,  # 默认学习率，通常比A2C更小
    buffer_size=100000,  # 经验回放缓冲区大小
    learning_starts=1000,  # 开始学习前的步数
    batch_size=32,  # 批量大小
    tau=1.0,  # 目标网络软更新的系数
    gamma=0.99,  # 折扣因子
    train_freq=4,  # 训练的频率（每4步训练一次）
    target_update_interval=1000,  # 目标网络更新的频率
    exploration_fraction=0.1,  # 探索率衰减部分
    exploration_final_eps=0.02,  # 最小探索率
    max_grad_norm=10,  # 梯度裁剪的最大范数
)


# 训练模型，并将TensorBoard日志存储到log目录中
model.learn(total_timesteps=1000000, tb_log_name="DQN", callback=eval_callback)

# 保存模型
model.save("model/dqn_electric_vehicle")

# 加载模型（如果需要）
# model = DQN.load("dqn_electric_vehicle")

# 评估模型
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")

# 测试训练好的模型
obs = env.reset()
for i in range(100):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
        obs = env.reset()

Eval num_timesteps=1000, episode_reward=-8412.84 +/- 145.73
Episode length: 3.65 +/- 1.06
New best mean reward!
Eval num_timesteps=2000, episode_reward=-6936.26 +/- 2888.20
Episode length: 2.25 +/- 0.70
New best mean reward!
Eval num_timesteps=3000, episode_reward=-7368.10 +/- 2402.49
Episode length: 2.25 +/- 0.54
Eval num_timesteps=4000, episode_reward=-6525.87 +/- 3248.09
Episode length: 2.00 +/- 0.00
New best mean reward!


In [3]:
import torch as th
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy

# 重新加载模型
model = DQN(
    "MlpPolicy", 
    env, 
    verbose=0, 
    tensorboard_log="./tensorboard/DQN/",
    #policy_kwargs=policy_kwargs,
    learning_rate=0.0005,  # 重新定义学习率
    buffer_size=100000,
    learning_starts=1000,
    batch_size=32,
    tau=1.0,
    gamma=0.99,
    train_freq=4,
    target_update_interval=1000,
    exploration_fraction=0.1,
    exploration_final_eps=0.02,
    max_grad_norm=10,
    device="cuda"  # 确保在GPU上训练
)

# 加载之前训练的最佳模型参数
model.set_parameters("model/dqn_retrained_electric_vehicle.zip")  # best_model 是之前保存的模型路径


# 开始继续训练
log_name = "DQN_Retrain"
model.learn(
    total_timesteps=500000,  # 继续训练200,000步
    tb_log_name=log_name,
    callback=eval_callback
)

# 保存重新训练的模型
model.save("model/dqn_retrained_electric_vehicle")

# 评估再训练后的模型表现
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"After retraining: mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

# 测试训练好的模型
obs = env.reset()
for i in range(100):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
        obs = env.reset()


Eval num_timesteps=1000, episode_reward=-5440.88 +/- 4307.78
Episode length: 7.10 +/- 4.40
New best mean reward!
Eval num_timesteps=2000, episode_reward=-4466.34 +/- 4415.86
Episode length: 6.00 +/- 4.39
New best mean reward!
Eval num_timesteps=3000, episode_reward=-7225.00 +/- 3501.53
Episode length: 9.15 +/- 3.15
Eval num_timesteps=4000, episode_reward=-7588.09 +/- 3168.74
Episode length: 8.95 +/- 3.77
Eval num_timesteps=5000, episode_reward=-6178.12 +/- 3974.14
Episode length: 6.80 +/- 4.23
Eval num_timesteps=6000, episode_reward=-6281.41 +/- 4033.77
Episode length: 7.30 +/- 4.23
Eval num_timesteps=7000, episode_reward=-9038.01 +/- 101.21
Episode length: 10.90 +/- 0.44
Eval num_timesteps=8000, episode_reward=-8471.67 +/- 1945.74
Episode length: 9.30 +/- 2.97
Eval num_timesteps=9000, episode_reward=-5985.93 +/- 3893.44
Episode length: 6.00 +/- 3.95
Eval num_timesteps=10000, episode_reward=-5827.77 +/- 4192.74
Episode length: 7.00 +/- 4.46
Eval num_timesteps=11000, episode_reward=-573

In [4]:
model.learn(
    total_timesteps=200000,  # 继续训练200,000步
    tb_log_name=log_name,
    callback=eval_callback
)
# 保存重新训练的模型
model.save("model/dqn_retrained_electric_vehicle1")

Eval num_timesteps=1000, episode_reward=-2266.95 +/- 3834.24
Episode length: 3.75 +/- 3.74
Eval num_timesteps=2000, episode_reward=-553.80 +/- 1963.85
Episode length: 2.70 +/- 2.30
Eval num_timesteps=3000, episode_reward=-477.86 +/- 1952.12
Episode length: 2.15 +/- 2.24
Eval num_timesteps=4000, episode_reward=-477.87 +/- 1797.20
Episode length: 1.55 +/- 0.74
Eval num_timesteps=5000, episode_reward=-497.09 +/- 1980.43
Episode length: 2.15 +/- 2.37
Eval num_timesteps=6000, episode_reward=-950.62 +/- 2599.56
Episode length: 2.30 +/- 2.49
Eval num_timesteps=7000, episode_reward=-509.32 +/- 1977.95
Episode length: 2.10 +/- 2.23
Eval num_timesteps=8000, episode_reward=-1412.45 +/- 3112.03
Episode length: 2.95 +/- 2.89
Eval num_timesteps=9000, episode_reward=-446.83 +/- 1836.46
Episode length: 1.75 +/- 1.18
Eval num_timesteps=10000, episode_reward=-1295.42 +/- 2997.57
Episode length: 2.25 +/- 1.41
Eval num_timesteps=11000, episode_reward=-1713.46 +/- 3377.72
Episode length: 2.30 +/- 2.24
Eval

In [5]:
model.learn(
    total_timesteps=400000,  # 继续训练200,000步
    tb_log_name=log_name,
    callback=eval_callback
)
# 保存重新训练的模型
model.save("model/dqn_retrained_electric_vehicle3")

Eval num_timesteps=1000, episode_reward=-28.24 +/- 119.18
Episode length: 2.55 +/- 1.40
Eval num_timesteps=2000, episode_reward=-9.53 +/- 103.10
Episode length: 2.00 +/- 1.22
Eval num_timesteps=3000, episode_reward=-436.16 +/- 1950.48
Episode length: 2.95 +/- 2.09
Eval num_timesteps=4000, episode_reward=-421.24 +/- 1791.48
Episode length: 1.90 +/- 0.99
Eval num_timesteps=5000, episode_reward=-459.08 +/- 1944.34
Episode length: 2.40 +/- 2.22
Eval num_timesteps=6000, episode_reward=-398.62 +/- 1797.50
Episode length: 2.00 +/- 1.14
Eval num_timesteps=7000, episode_reward=20.79 +/- 133.35
Episode length: 2.20 +/- 0.98
Eval num_timesteps=8000, episode_reward=-353.87 +/- 1782.82
Episode length: 2.25 +/- 1.09
Eval num_timesteps=9000, episode_reward=-401.71 +/- 1851.86
Episode length: 2.65 +/- 1.59
Eval num_timesteps=10000, episode_reward=-378.46 +/- 1778.20
Episode length: 2.45 +/- 1.20
Eval num_timesteps=11000, episode_reward=40.81 +/- 114.22
Episode length: 2.35 +/- 1.11
Eval num_timesteps=

MemoryError: 

In [6]:
model.save("model/dqn_retrained_electric_vehicle3")